In [1]:
import pandas as pd
import numpy as np
from string import ascii_lowercase
import itertools
import string
from collections import defaultdict

pd.set_option('display.max_columns', 50)

### Board

In [56]:
connect_n = 4
board_height = 6
board_width = 7 # max 26

In [57]:
columns = [column for column in ascii_lowercase[:board_width]]
rows = [row for row in range(board_height)]

In [58]:
X_cols = []
for column in ascii_lowercase[:board_width]:
    for row in range(board_height):
        X_cols.append(column + str(row))

### Winning positions

In [59]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1
win_types = ['up', 'across', 'diag /', 'diag \\']
win_type = 'diag \\'

In [122]:
def rows_and_columns_checked(win_type, all_rows, all_columns):
    win_type_positions = {}
    limit_top_rows = all_rows[:-connect_n + 1]
    limit_bottom_rows = all_rows[connect_n - 1:]
    limit_right_columns =  all_columns[:-connect_n + 1]
    
    # create dictionary of win types and a tuple of the rows and columns checked for each win type
    for win_type in win_types:
        if win_type == 'up':
            rows_checked, columns_checked = limit_top_rows, all_columns
        elif win_type == 'across':
            rows_checked, columns_checked = all_rows, limit_right_columns
        elif win_type == 'diag /':
            rows_checked, columns_checked = limit_top_rows, limit_right_columns
        elif win_type == 'diag \\':
            rows_checked, columns_checked = limit_bottom_rows, limit_right_columns
        else:
            raise ValueError('not a valid win type')
        win_type_positions[win_type] = rows_checked, columns_checked

    # create dictionary of individual row/column positions and a list of their possible win types 
    positions_win_types_to_check = defaultdict(list)
    for win_type, rows_and_columns_to_check in win_type_positions.iteritems():
        for positions in itertools.product(rows_and_columns_to_check[0],rows_and_columns_to_check[1]):
            row = positions[0]
            column = positions[1]
            position = column + str(row)
            positions_win_types_to_check[position, (row, column)].append(win_type)
    return positions_win_types_to_check

In [123]:
def win_type_angle(win_type, row_checked, column_checked):

    # list row and column positions
    if win_type == 'up':
        angle_column_positions = list(column_checked) * connect_n
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'across':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(str(row_checked)) * connect_n
    elif win_type == 'diag /':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'diag \\':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked - connect_n, -1)))
    else:
        raise ValueError('not a valid win type')

    # combine postitions
    return [column + row for column, row in zip(angle_column_positions, angle_row_positions)]

In [124]:
def get_winning_positions(positions_win_types_to_check):
    winning_positions = defaultdict(list)
    for position, position_win_types in positions_win_types_to_check.iteritems():
        row = position[1][0]
        column = position[1][1]
        for win_type in position_win_types:
            positions_to_check = win_type_angle(win_type, row, column)
            winning_positions[position[0]].append((win_type, positions_to_check))
    return winning_positions

In [125]:
positions_win_types_to_check = rows_and_columns_checked(win_type, rows, columns)

In [126]:
winning_positions = get_winning_positions(positions_win_types_to_check)

### Check for win

In [162]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1

In [185]:
winning_info = {}
for idx, board_positions in X_rand[:10].iterrows():
    winning_info[idx] = 0
    for position, token in board_positions[board_positions.isin([-1, 1])].iteritems():
        for win_type, positions in winning_positions[position]:
            if board_positions[positions].map(lambda x: x == token).all():
                winning_info[idx] = token
                print str(idx) + ' {} | wins in {} position | with a {} win type'.format(token, position, win_type)

1 -1 | wins in b5 position | with a diag \ win type
1 -1 | wins in c4 position | with a diag \ win type
1 -1 | wins in d3 position | with a diag \ win type
2 -1 | wins in d3 position | with a across win type
2 -1 | wins in d5 position | with a diag \ win type
2 -1 | wins in g2 position | with a up win type
3 1 | wins in a5 position | with a diag \ win type
3 1 | wins in a5 position | with a across win type
3 1 | wins in b4 position | with a diag \ win type
5 -1 | wins in a0 position | with a across win type
5 1 | wins in e0 position | with a up win type
5 -1 | wins in f2 position | with a up win type
6 -1 | wins in b1 position | with a up win type
8 1 | wins in a5 position | with a diag \ win type
9 -1 | wins in c0 position | with a across win type
9 1 | wins in f1 position | with a up win type


In [193]:
initial_example = X_rand.loc[6:8]

In [194]:
initial_example

,a0,a1,a2,a3,a4,a5,b0,b1,b2,b3,b4,b5,c0,c1,c2,c3,c4,c5,d0,d1,d2,d3,d4,d5,e0,e1,e2,e3,e4,e5,f0,f1,f2,f3,f4,f5,g0,g1,g2,g3,g4,g5
6,1,1,1,-1,1,1,0,-1,-1,-1,-1,1,0,0,-1,1,1,0,1,0,1,0,1,-1,-1,1,1,-1,0,1,-1,1,-1,0,-1,-1,1,-1,1,0,1,-1
7,-1,-1,-1,1,1,-1,1,0,0,0,-1,0,1,0,1,1,0,-1,0,1,1,-1,-1,-1,1,-1,0,-1,1,1,-1,0,0,1,0,1,-1,0,0,-1,0,0
8,1,1,-1,1,-1,1,-1,1,1,-1,1,-1,1,-1,0,1,0,-1,1,1,1,-1,0,1,0,0,0,0,-1,0,1,0,-1,-1,1,1,0,-1,-1,-1,1,1


In [188]:
dict((k, winning_info[k]) for k in (6, 7, 8))

{6: -1, 7: 0, 8: 1}

In [186]:
winning_info[]

{0: 0, 1: -1, 2: -1, 3: 1, 4: 0, 5: -1, 6: -1, 7: 0, 8: 1, 9: 1}

In [164]:
X_rand.iloc[1]

a0   -1
a1    0
a2    1
a3    0
a4   -1
a5    1
b0    1
b1    0
b2    0
b3    0
b4   -1
b5   -1
c0    0
c1   -1
c2    1
c3   -1
c4   -1
c5    0
d0    0
d1    1
d2   -1
d3   -1
d4    0
d5    1
e0    1
e1    0
e2   -1
e3    1
e4    0
e5    1
f0    0
f1   -1
f2    0
f3    0
f4    1
f5   -1
g0   -1
g1    1
g2    1
g3    0
g4   -1
g5    1
Name: 1, dtype: int64